In [1]:
import numpy as np
import scipy.io as sio

from DataViewer import DataViewer

In [ ]:
# Load names
scan_name = 'Scan_4'
peak_pre = './DataOutput/Peaks_'
vol_pre = './DataOutput/Volume_'

peak_name = peak_pre + scan_name
vol_name = vol_pre + scan_name

In [3]:
# Load data
peak_dict = sio.loadmat(peak_name+'.mat')
vol_dict = sio.loadmat(vol_name+'.mat')

FileNotFoundError: [Errno 2] No such file or directory: 'Peaks_Scan_4.mat'

In [ ]:
print(peak_dict)
print(vol_dict)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Sep 15 10:32:40 2025', '__version__': '1.0', '__globals__': [], 'peaks': array([[860, 300, 822],
       [822, 424, 828],
       [132, 794, 812],
       ...,
       [  2,  44, 420],
       [ 76,  52, 546],
       [ 32,  50, 668]], shape=(680, 3), dtype=uint16)}
{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Sep 15 10:32:40 2025', '__version__': '1.0', '__globals__': []}


In [ ]:
dog = [2,0,1]
meow = tuple(dog)
print(dog,meow)

[2, 0, 1] (2, 0, 1)


In [ ]:
# Refine data
unordered_peaks = np.array(peak_dict.get('peaks'))

unordered_vol = np.array(vol_dict.get('dataFinal'))
print(unordered_vol)

perm = [2,0,1]
peaks = unordered_peaks[:,perm]

print(unordered_peaks[0]) # [y,x,z]
print(peaks[0]) # [822,860,300]

print(np.shape(peaks))

print(unordered_vol)
vol = np.transpose(unordered_vol, tuple(perm))


print(np.shape(vol))

print(np.shape(unordered_peaks))

None
[860 300 822]
[822 860 300]
(680, 3)
None


ValueError: axes don't match array

In [ ]:
>>> import numpy as np
>>> a = np.array([[10, 20, 30, 40, 50],
...               [ 6,  7,  8,  9, 10]])
>>> permutation = [0, 4, 1, 3, 2]
>>> idx = np.empty_like(permutation)
>>> idx[permutation] = np.arange(len(permutation))
>>> a[:, idx]  # return a rearranged copy
array([[10, 30, 50, 40, 20],
       [ 6,  8, 10,  9,  7]])
>>> a[:] = a[:, idx]  # in-place modification of a

In [ ]:
Cropped:(921,891,852) # [y,x,z]

peaks_csv = np.loadmat(file_name) # [y,x,z]
peaks_unordered = peaks_csv.tolist()
peaks = peaks_unordered[2,0,1] # [z,y,x]